In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import f, norm, pearsonr, ttest_rel

# Load data
study_name = "Preferences for redistribution"
specification_name = "default persona"
human_file = f"{study_name} human data labels anonymized.csv"
twin_file = f"{study_name} twins data labels anonymized.csv"
df_human = pd.read_csv(human_file, header=0, skiprows=[1, 2])
df_twin = pd.read_csv(twin_file, header=0, skiprows=[1, 2])

# remove observations if their completion time was less than 50% of the median survey duration.
# Compute 50%-of-median thresholds
median_human = df_human["Duration (in seconds)"].median()
threshold_human = 0.5 * median_human
median_twin = df_twin["Duration (in seconds)"].median()
threshold_twin = 0.5 * median_twin
# Identify which TWIN_IDs would be removed
removed_human = df_human.loc[df_human["Duration (in seconds)"] < threshold_human, "TWIN_ID"]
removed_twin = df_twin.loc[df_twin["Duration (in seconds)"] < threshold_twin, "TWIN_ID"]
set_human = set(removed_human)
set_twin = set(removed_twin)
# Compare the two sets
print(f"Human-sample removals due to response time ({len(set_human)} IDs):\n", set_human)
print(f"Twin-sample removals due to response time ({len(set_twin)} IDs):\n", set_twin)
print("Sets equal? ", set_human == set_twin)
print("In human only:", set_human - set_twin)
print("In twin only: ", set_twin - set_human)

# Actually filter out those rows
df_human_filtered = df_human[df_human["Duration (in seconds)"] >= threshold_human].copy()
df_twin_filtered = df_twin[df_twin["Duration (in seconds)"] >= threshold_twin].copy()
# inspect sizes
print("After filtering:")
print(" df_human:", df_human.shape, "→", df_human_filtered.shape)
print(" df_twin: ", df_twin.shape, "→", df_twin_filtered.shape)

# remove those failing the attention check question in the survey. Specifically, this questions asks: "To ensure you're paying attention, please select “Don’t know” as your answer to this question:
# Identify who failed in df_human and df_twin
fail_human = df_human_filtered.loc[df_human_filtered["ATTENTION_CHECK"] != "Don’t know", "TWIN_ID"]
fail_twin = df_twin_filtered.loc[df_twin_filtered["ATTENTION_CHECK"] != "Don’t know", "TWIN_ID"]
# Print the sets of failing IDs
set_fail_human = set(fail_human)
set_fail_twin = set(fail_twin)
print("Failed attention in human (n={}):".format(len(set_fail_human)), set_fail_human)
print("Failed attention in twin  (n={}):".format(len(set_fail_twin)), set_fail_twin)
# Remove everyone who failed in the *human* sample, from both dfs
ids_to_remove = set_fail_human
df_human_clean = df_human_filtered.loc[~df_human_filtered["TWIN_ID"].isin(ids_to_remove)].copy()

df_twin_clean = df_twin_filtered.loc[~df_twin_filtered["TWIN_ID"].isin(ids_to_remove)].copy()
# Quick sanity check
print("\nAfter attention‐check filtering:")
print(" df_human: {} → {}".format(df_human_filtered.shape, df_human_clean.shape))
print(" df_twin:  {} → {}".format(df_twin_filtered.shape, df_twin_clean.shape))


df_human = df_human_clean
df_twin = df_twin_clean


# add new columns with relevant variables coded

redistribution1_map = {
    "1 - I strongly agree the government should improve living standards": 5,
    "2": 4,
    "3 - I agree with both answers": 3,
    "4": 2,
    "5 - I strongly agree that people should take care of themselves": 1,
    # any other value (e.g. "Don't know") will become NaN
}

redistribution2_map = {
    "Very willing": 5,
    "Fairly willing": 4,
    "Neither willing nor unwilling": 3,
    "Fairly unwilling": 2,
    "Very unwilling": 1,
    # e.g. "Donâ€™t know/Can't choose" → NaN
}

fairness_map = {
    "Would take advantage of you": 3,
    "It depends": 2,
    "Would try to be fair": 1,
    # "Don’t know" → NaN
}

workluck_map = {
    "Hard work most important": 1,
    "Hard work and luck are equally important": 2,
    "Luck or help from other people most important": 3,
    # "Don't know" → NaN
}

father_educ_map = {
    "Less than high school": 10,
    "High school graduate": 12,
    "Some college, no degree": 13,
    "Associate's degree": 14,
    "College graduate / some postgraduate": 16,
    "Postgraduate degree": 18,
}

trust_map = {
    "Can trust": 3,
    "Depends": 2,
    "Can't be too careful": 1,
    # "Don’t know" → NaN
}


for df in (df_human, df_twin):
    df["Redistribution 1"] = df["GOVT_RESPONSIBILITY"].map(redistribution1_map)
    df["Redistribution 2"] = df["HLTHTAX"].map(redistribution2_map)
    df["Fairness"] = df["FAIR"].map(fairness_map)
    df["Work vs Luck"] = df["GETAHEAD"].map(workluck_map)
    df["Father education"] = df["FAEDUC"].map(father_educ_map)
    df["Trust"] = df["TRUST_GSS"].map(trust_map)

# check results:
pairs = [
    ("GOVT_RESPONSIBILITY", "Redistribution 1"),
    ("HLTHTAX", "Redistribution 2"),
    ("FAIR", "Fairness"),
    ("GETAHEAD", "Work vs Luck"),
    ("FAEDUC", "Father education"),
    ("TRUST_GSS", "Trust"),
]

for df_name, df in [("Human", df_human), ("Twin", df_twin)]:
    print(f"\n==== {df_name} sample ====\n")
    for orig, new in pairs:
        print(f"– {orig} vs {new} –")
        ct = pd.crosstab(df[orig], df[new], dropna=False)
        print(ct)
        print()


# define relevant columns:
# condition variable names:
condition_vars = [""]
# Check if we have a real condition var
if condition_vars and condition_vars[0].strip():
    cond = condition_vars[0]
    cond_h = f"{cond}_human"
    cond_t = f"{cond}_twin"
    cond_exists = True
else:
    cond_exists = False

# #raw responses:
# raw_vars = ['Redistribution 1','Redistribution 2','Fairness','Work vs Luck','Father education','Trust']
# raw_vars_min = [1,1,1,1,10,1]
# raw_vars_max = [5,5,3,3,18,3]
# #raw responses: domain=social?
# raw_vars_social=[1,1,1,1,0,1]
# raw_vars_social_map = dict(zip(raw_vars, raw_vars_social))
# #raw responses: domain=cognitive?
# raw_vars_cognitive=[0]*6
# raw_vars_cognitive_map = dict(zip(raw_vars, raw_vars_cognitive))
# #raw responses: replicating know human bias?
# raw_vars_known=[0]*6
# raw_vars_known_map = dict(zip(raw_vars, raw_vars_known))
# #raw responses: preference measure?
# raw_vars_pref=[1,1,0,0,0,0]
# raw_vars_pref_map = dict(zip(raw_vars, raw_vars_pref))
# #raw responses: stimuli dependent?
# raw_vars_stim=[0]*6
# raw_vars_stim_map = dict(zip(raw_vars, raw_vars_stim))

# DVs:
DV_vars = [
    "Redistribution 1",
    "Redistribution 2",
    "Fairness",
    "Work vs Luck",
    "Father education",
    "Trust",
]
DV_vars_min = [1, 1, 1, 1, 10, 1]
DV_vars_max = [5, 5, 3, 3, 18, 3]
# DVs: domain=social?
DV_vars_social = [1, 1, 1, 1, 0, 1]
DV_vars_social_map = dict(zip(DV_vars, DV_vars_social))
# DVs: domain=cognitive?
DV_vars_cognitive = [0] * 6
DV_vars_cognitive_map = dict(zip(DV_vars, DV_vars_cognitive))
# DVs: replicating know human bias?
DV_vars_known = [0] * 6
DV_vars_known_map = dict(zip(DV_vars, DV_vars_known))
# DVs: preference measure?
DV_vars_pref = [1, 1, 0, 0, 0, 0]
DV_vars_pref_map = dict(zip(DV_vars, DV_vars_pref))
# DVs: stimuli dependent?
DV_vars_stim = [0] * 6
DV_vars_stim_map = dict(zip(DV_vars, DV_vars_stim))
# DVs: knowledge question?
DV_vars_know = [0] * 6
DV_vars_know_map = dict(zip(DV_vars, DV_vars_know))
# DVs: political question?
DV_vars_politics = [1, 1, 0, 0, 0, 0]
DV_vars_politics_map = dict(zip(DV_vars, DV_vars_politics))


# merging key
merge_key = ["TWIN_ID"]

# Merge on TWIN_ID
df = pd.merge(df_human, df_twin, on=merge_key, suffixes=("_human", "_twin"))

# Fix dtypes
# for var in raw_vars + DV_vars:
for var in DV_vars:
    df[f"{var}_human"] = pd.to_numeric(df[f"{var}_human"], errors="coerce")
    df[f"{var}_twin"] = pd.to_numeric(df[f"{var}_twin"], errors="coerce")

# build min/max maps from both raw_vars and DV_vars
min_map = {v: mn for v, mn in zip(DV_vars, DV_vars_min)}
# min_map = {v: mn for v, mn in zip(raw_vars,      raw_vars_min)}
# min_map.update({v: mn for v, mn in zip(DV_vars,   DV_vars_min)})

max_map = {v: mx for v, mx in zip(DV_vars, DV_vars_max)}
# max_map = {v: mx for v, mx in zip(raw_vars,      raw_vars_max)}
# max_map.update({v: mx for v, mx in zip(DV_vars,   DV_vars_max)})

# now add _min and _max columns for every variable in the union
for var in min_map:
    df[f"{var}_min"] = min_map[var]
    df[f"{var}_max"] = max_map[var]

# Compute results
results = []
# for var in raw_vars:
#     col_h = f"{var}_human"
#     col_t = f"{var}_twin"
#     min_col = f"{var}_min"
#     max_col = f"{var}_max"
#     if cond_exists:
#         cols = [col_h, col_t, cond_h, cond_t,min_col,max_col]
#     else:
#         cols = [col_h, col_t,min_col,max_col]
#     pair = (
#     df[cols]
#       .dropna(subset=[col_h, col_t])
#     )
#     min_val = pair[min_col].iloc[0]
#     max_val = pair[max_col].iloc[0]
#     n    = len(pair)
#     if n >= 4:
#         r, _    = pearsonr(pair[col_h], pair[col_t])
#         z_f     = np.arctanh(r)
#         se      = 1 / np.sqrt(n - 3)
#         z_crit  = norm.ppf(0.975)
#         lo_z, hi_z = z_f - z_crit*se, z_f + z_crit*se
#         lo_r, hi_r = np.tanh(lo_z), np.tanh(hi_z)
#         z_score    = z_f / se
#         # Accuracy = mean absolute diff / range
#         if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
#             accuracy = np.nan
#         else:
#             # compute mean absolute difference
#             abs_diff      = np.abs(pair[col_h] - pair[col_t])
#             mean_abs_diff = abs_diff.mean()
#             accuracy      = 1 - mean_abs_diff / (max_val - min_val)

#         mean_h = pair[col_h].mean()
#         mean_t = pair[col_t].mean()

#         # Paired t‐test
#         t_stat, p_val = ttest_rel(pair[col_h], pair[col_t])

#         std_h = pair[col_h].std(ddof=1)
#         std_t = pair[col_t].std(ddof=1)

#          # F‐test for equal variances
#         df1 = df2 = n - 1
#         f_stat = (std_h**2 / std_t**2) if std_t>0 else np.nan

#         # two‐tailed p‐value:
#         if not np.isnan(f_stat):
#             p_f = 2 * min(f.cdf(f_stat, df1, df2),
#                           1 - f.cdf(f_stat, df1, df2))
#         else:
#             p_f = np.nan

#         # Effect sizes (Cohen's d) across conditions
#         #    For humans:
#         if cond_exists and len(pair)>3:
#             levels_h = pair[cond_h].unique()
#             if len(levels_h) == 2:
#                 g1 = pair.loc[pair[cond_h]==levels_h[0], col_h]
#                 g2 = pair.loc[pair[cond_h]==levels_h[1], col_h]
#                 n1, n2 = len(g1), len(g2)
#                 # pooled sd
#                 s_pool = np.sqrt(((n1-1)*g1.var(ddof=1)+(n2-1)*g2.var(ddof=1)) / (n1+n2-2))
#                 d_human = (g1.mean() - g2.mean()) / s_pool if s_pool>0 else np.nan
#             else:
#                 d_human = np.nan
#         else:
#             d_human = np.nan

#         #    For twins:
#         if cond_exists and len(pair)>3:
#             levels_t = pair[cond_t].unique()
#             if cond_exists and len(levels_t) == 2:
#                 g1 = pair.loc[pair[cond_t]==levels_t[0], col_t]
#                 g2 = pair.loc[pair[cond_t]==levels_t[1], col_t]
#                 n1, n2 = len(g1), len(g2)
#                 s_pool = np.sqrt(((n1-1)*g1.var(ddof=1)+(n2-1)*g2.var(ddof=1)) / (n1+n2-2))
#                 d_twin = (g1.mean() - g2.mean()) / s_pool if s_pool>0 else np.nan
#             else:
#                 d_twin = np.nan
#         else:
#             d_twin = np.nan
#     else:
#         r = lo_r = hi_r = z_score = accuracy = mean_h = mean_t = t_stat = p_val = std_h = std_t = f_stat = p_f = np.nan
#         d_human = d_twin = np.nan


#     results.append({
#         'study name': study_name,
#         'variable name': var,
#         'variable type (raw response/DV)':     'raw',
#         'correlation between the responses from humans vs. their twins':        r,
#         'CI_lower': lo_r,
#         'CI_upper': hi_r,
#         'z-score for correlation between humans vs. their twins':  z_score,
#         'accuracy between humans vs. their twins': accuracy,
#         'mean_human': mean_h,
#         'mean_twin': mean_t,
#         'paired t-test t-stat': t_stat,
#         'paired t-test p-value': p_val,
#         'std_human': std_h,
#         'std_twin': std_t,
#         'variance test F-stat': f_stat,
#         'variance test p-value': p_f,
#         'effect size based on human': d_human,
#         'effect size based on twin': d_twin,
#         'domain=social?':raw_vars_social_map.get(var, np.nan),
#         'domain=cognitive?':raw_vars_cognitive_map.get(var, np.nan),
#         'replicating know human bias?':raw_vars_known_map.get(var, np.nan),
#         'preference measure?':raw_vars_pref_map.get(var, np.nan),
#         'stimuli dependent?':raw_vars_stim_map.get(var, np.nan),
#         'sample size':        n
#     })

for var in DV_vars:
    col_h = f"{var}_human"
    col_t = f"{var}_twin"
    min_col = f"{var}_min"
    max_col = f"{var}_max"
    if cond_exists:
        cols = [col_h, col_t, cond_h, cond_t, min_col, max_col]
    else:
        cols = [col_h, col_t, min_col, max_col]
    pair = df[cols].dropna(subset=[col_h, col_t])
    min_val = pair[min_col].iloc[0]
    max_val = pair[max_col].iloc[0]
    n = len(pair)
    if n >= 4:
        r, _ = pearsonr(pair[col_h], pair[col_t])
        z_f = np.arctanh(r)
        se = 1 / np.sqrt(n - 3)
        z_crit = norm.ppf(0.975)
        lo_z, hi_z = z_f - z_crit * se, z_f + z_crit * se
        lo_r, hi_r = np.tanh(lo_z), np.tanh(hi_z)
        z_score = z_f / se
        # Accuracy = mean absolute diff / range
        if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
            accuracy = np.nan
        else:
            # compute mean absolute difference
            abs_diff = np.abs(pair[col_h] - pair[col_t])
            mean_abs_diff = abs_diff.mean()
            accuracy = 1 - mean_abs_diff / (max_val - min_val)

        mean_h = pair[col_h].mean()
        mean_t = pair[col_t].mean()

        # Paired t‐test
        t_stat, p_val = ttest_rel(pair[col_h], pair[col_t])

        std_h = pair[col_h].std(ddof=1)
        std_t = pair[col_t].std(ddof=1)

        # F‐test for equal variances
        df1 = df2 = n - 1
        f_stat = (std_h**2 / std_t**2) if std_t > 0 else np.nan
        # two‐tailed p‐value:
        if not np.isnan(f_stat):
            p_f = 2 * min(f.cdf(f_stat, df1, df2), 1 - f.cdf(f_stat, df1, df2))
        else:
            p_f = np.nan

        # Effect sizes (Cohen's d) across conditions
        #    For humans:
        if cond_exists and len(pair) > 3:
            levels_h = pair[cond_h].unique()
            if len(levels_h) == 2:
                g1 = pair.loc[pair[cond_h] == levels_h[0], col_h]
                g2 = pair.loc[pair[cond_h] == levels_h[1], col_h]
                n1, n2 = len(g1), len(g2)
                # pooled sd
                s_pool = np.sqrt(
                    ((n1 - 1) * g1.var(ddof=1) + (n2 - 1) * g2.var(ddof=1)) / (n1 + n2 - 2)
                )
                d_human = (g1.mean() - g2.mean()) / s_pool if s_pool > 0 else np.nan
            else:
                d_human = np.nan
        else:
            d_human = np.nan

        #    For twins:
        if cond_exists and len(pair) > 3:
            levels_t = pair[cond_t].unique()
            if cond_exists and len(levels_t) == 2:
                g1 = pair.loc[pair[cond_t] == levels_t[0], col_t]
                g2 = pair.loc[pair[cond_t] == levels_t[1], col_t]
                n1, n2 = len(g1), len(g2)
                s_pool = np.sqrt(
                    ((n1 - 1) * g1.var(ddof=1) + (n2 - 1) * g2.var(ddof=1)) / (n1 + n2 - 2)
                )
                d_twin = (g1.mean() - g2.mean()) / s_pool if s_pool > 0 else np.nan
            else:
                d_twin = np.nan
        else:
            d_twin = np.nan
    else:
        r = lo_r = hi_r = z_score = accuracy = mean_h = mean_t = t_stat = p_val = std_h = std_t = (
            f_stat
        ) = p_f = np.nan
        d_human = d_twin = np.nan

    results.append(
        {
            "study name": study_name,
            "persona specification": specification_name,
            "variable name": var,
            #        'variable type (raw response/DV)':     'DV',
            "correlation between the responses from humans vs. their twins": r,
            "CI_lower": lo_r,
            "CI_upper": hi_r,
            "z-score for correlation between humans vs. their twins": z_score,
            "accuracy between humans vs. their twins": accuracy,
            "mean_human": mean_h,
            "mean_twin": mean_t,
            "paired t-test t-stat": t_stat,
            "paired t-test p-value": p_val,
            "std_human": std_h,
            "std_twin": std_t,
            "variance test F-stat": f_stat,
            "variance test p-value": p_f,
            "effect size based on human": d_human,
            "effect size based on twin": d_twin,
            "domain=social?": DV_vars_social_map.get(var, np.nan),
            "domain=cognitive?": DV_vars_cognitive_map.get(var, np.nan),
            "replicating know human bias?": DV_vars_known_map.get(var, np.nan),
            "preference measure?": DV_vars_pref_map.get(var, np.nan),
            "stimuli dependent?": DV_vars_stim_map.get(var, np.nan),
            "knowledge question?": DV_vars_know_map.get(var, np.nan),
            "political question?": DV_vars_politics_map.get(var, np.nan),
            "sample size": n,
        }
    )

# results DataFrame
corr_df = pd.DataFrame(results)
print(corr_df)

# save output as csv - unit of observation is comparison between humans and twins:
out_file = f"{study_name} {specification_name} meta analysis.csv"
corr_df.to_csv(out_file, index=False)


#####participant-level data:
def make_long(df, respondent_type):
    # pick off TWIN_ID + the DVs, then melt
    long = df[["TWIN_ID"] + DV_vars].melt(
        id_vars="TWIN_ID", value_vars=DV_vars, var_name="variable_name", value_name="value"
    )

    long["respondent_type"] = respondent_type
    long["study_name"] = study_name
    long["specification_name"] = specification_name
    return long


# build the two halves
long_h = make_long(df_human, "human")
long_t = make_long(df_twin, "twin")

# stack them
df_long = pd.concat([long_h, long_t], ignore_index=True)

print(df_long.head())
# save output as csv - unit of observation is TWIN_ID:
out_file = f"{study_name} {specification_name} meta analysis individual level.csv"
df_long.to_csv(out_file, index=False)

print("done")

C:\Users\ot2107\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Human-sample removals due to response time (35 IDs):
 {1025, 1291, 13, 21, 537, 410, 924, 1308, 542, 799, 928, 1182, 1184, 1567, 1954, 806, 1577, 44, 47, 1337, 1085, 1981, 67, 1347, 72, 336, 998, 1383, 618, 880, 1013, 246, 1016, 1530, 1915}
Twin-sample removals due to response time (35 IDs):
 {1025, 1291, 13, 21, 537, 410, 924, 1308, 542, 799, 928, 1182, 1184, 1567, 1954, 806, 1577, 44, 47, 1337, 1085, 1981, 67, 1347, 72, 336, 998, 1383, 618, 880, 1013, 246, 1016, 1530, 1915}
Sets equal?  True
In human only: set()
In twin only:  set()
After filtering:
 df_human: (1198, 35) → (1163, 35)
 df_twin:  (1198, 35) → (1163, 35)
Failed attention in human (n=0): set()
Failed attention in twin  (n=0): set()

After attention‐check filtering:
 df_human: (1163, 35) → (1163, 35)
 df_twin:  (1163, 35) → (1163, 35)

==== Human sample ====

– GOVT_RESPONSIBILITY vs Redistribution 1 –
Redistribution 1                                    1.0  2.0  3.0  4.0  5.0  \
GOVT_RESPONSIBILITY                       